In [1]:
%pip uninstall -y lsdb hipscat

# %pip install git+http://hipscat.rubin.science
# %pip install git+http://lsdb.rubin.science

%pip install lsdb

Found existing installation: lsdb 0.2.7.dev31+g79080ad
Uninstalling lsdb-0.2.7.dev31+g79080ad:
  Successfully uninstalled lsdb-0.2.7.dev31+g79080ad
Found existing installation: hipscat 0.3.6.dev2+g97643b6
Uninstalling hipscat-0.3.6.dev2+g97643b6:
  Successfully uninstalled hipscat-0.3.6.dev2+g97643b6
Note: you may need to restart the kernel to use updated packages.
  Using cached lsdb-0.2.7-py3-none-any.whl.metadata (5.9 kB)
  Using cached hipscat-0.3.7-py3-none-any.whl.metadata (6.0 kB)
Using cached lsdb-0.2.7-py3-none-any.whl (65 kB)
Using cached hipscat-0.3.7-py3-none-any.whl (77 kB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
WITH_GAIA = True

In [2]:
catalog_name = 'ztf_ps1'
if WITH_GAIA:
    catalog_name += '_gaia'

In [3]:
%%time

from pathlib import Path

import dask.distributed
import lsdb
import pyarrow as pa
import pyarrow.dataset
import pyarrow.parquet

CPU times: user 13 s, sys: 7.9 s, total: 20.9 s
Wall time: 59.3 s


In [4]:
%%time

HIPSCAT_PATH = Path('/ocean/projects/phy210048p/shared/hipscat/catalogs/')

PS1_BANDS = 'riz'

ztf_dr17_coord = lsdb.read_hipscat(
    'hipscat/ztf_dr17_coord',
    margin_cache='hipscat/ztf_dr17_coord_2arcsec',
).query('filter == 2')  # r-band only
gaia_distances = lsdb.read_hipscat(
    HIPSCAT_PATH / 'gaia_dr3' / 'gaia_edr3_distances',
    margin_cache = str(HIPSCAT_PATH / 'gaia_dr3' / 'gaia_edr3_distances_10arcs'),
    # Remove RA and Dec - we have them from the main Gaia catalog
    columns=[
        'source_id',
        'r_med_geo', 'r_lo_geo', 'r_hi_geo',
        'r_med_photogeo', 'r_lo_photogeo', 'r_hi_photogeo',
        'flag',
    ]
)
gaia = lsdb.read_hipscat(
    HIPSCAT_PATH / 'gaia_dr3' / 'gaia',
    # We must do str() due to
    # https://github.com/astronomy-commons/lsdb/issues/380
    margin_cache=str(HIPSCAT_PATH / 'gaia_dr3' / 'gaia_10arcs'),
    columns=[
        'ra', 'dec',
        'source_id',
        'ruwe',
        'parallax', 'parallax_over_error',
        'pmra', 'pmdec', 'pmra_error', 'pmdec_error',
        'teff_gspphot', 'teff_gspphot_lower', 'teff_gspphot_upper',
        'logg_gspphot', 'logg_gspphot_lower', 'logg_gspphot_upper',
        # 'ag_gspphot', 'ag_gspphot_lower', 'ag_gspphot_upper',
    ],
).query(
    "teff_gspphot_upper < 3800"
    # " and ruwe < 1.4"
    
    # "parallax_over_error > 10.0"
    # " and teff_gspphot_upper < 3800"
    # " and (teff_gspphot_upper - teff_gspphot_lower) < 400"
    # " and logg_gspphot_lower > 4.5"
    # " and (logg_gspphot_upper - logg_gspphot_lower) < 0.2"
)
panstarrs = lsdb.read_hipscat(
    HIPSCAT_PATH / 'ps1' / 'ps1_otmo',
    margin_cache=str(HIPSCAT_PATH / 'ps1' / 'ps1_otmo_10arcs'),
    columns=
        ['raMean', 'decMean']
        + [f'{b}MeanPSFMag' for b in PS1_BANDS]
        + [f'{b}MeanPSFMagErr' for b in PS1_BANDS],
).query(
    "((rMeanPSFMag - iMeanPSFMag) + (rMeanPSFMagErr + iMeanPSFMagErr)) > 0.42"
    " and ((iMeanPSFMag - zMeanPSFMag) + (iMeanPSFMagErr + zMeanPSFMagErr)) > 0.23"
    " and rMeanPSFMagErr < 0.1 and iMeanPSFMagErr < 0.1 and zMeanPSFMagErr < 0.1"
)

/ocean/projects/phy210048p/malanche/ztfdr17-psdr2-gaiadr3/cenv/lib/python3.12/site-packages/hipscat/io/file_io/file_io.py:308: UserWarning: point_map.fits file written in RING ordering, due to https://github.com/astronomy-commons/hipscat/issues/271. Converting to NESTED.
  warnings.warn(
/ocean/projects/phy210048p/malanche/ztfdr17-psdr2-gaiadr3/cenv/lib/python3.12/site-packages/hipscat/io/file_io/file_io.py:308: UserWarning: point_map.fits file written in RING ordering, due to https://github.com/astronomy-commons/hipscat/issues/271. Converting to NESTED.
  warnings.warn(


CPU times: user 2.21 s, sys: 290 ms, total: 2.5 s
Wall time: 4.12 s


In [5]:
%%time

if WITH_GAIA:
    catalog = ztf_dr17_coord.crossmatch(
        gaia,
        radius_arcsec=1,
        n_neighbors=1,
        suffixes=['', '_gaia'],
    )
else:
    catalog = ztf_dr17_coord
    
catalog = catalog.crossmatch(
    panstarrs,
    radius_arcsec=1,
    n_neighbors=1,
    suffixes=['', ''],
)

# We must do join after cross-match to not lose margin cache due to
# https://github.com/astronomy-commons/lsdb/issues/382
if WITH_GAIA:
    catalog = catalog.join(gaia_distances, left_on='source_id_gaia', right_on='source_id', suffixes=['', '_edr3dist'])

CPU times: user 15min 2s, sys: 21.4 s, total: 15min 24s
Wall time: 15min 26s


In [8]:
%%time

with dask.distributed.Client(n_workers=16) as client:
    display(client)
    catalog._ddf.to_parquet(catalog_name)

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 16
Total threads: 128,Total memory: 247.07 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:36643,Workers: 16
Dashboard: http://127.0.0.1:8787/status,Total threads: 128
Started: Just now,Total memory: 247.07 GiB
Comm: tcp://127.0.0.1:39389,Total threads: 8
Dashboard: http://127.0.0.1:36501/status,Memory: 15.44 GiB
Nanny: tcp://127.0.0.1:44627,


/ocean/projects/phy210048p/malanche/ztfdr17-psdr2-gaiadr3/cenv/lib/python3.12/site-packages/distributed/client.py:3164: UserWarning: Sending large graph of size 39.01 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
2024-07-26 11:36:39,049 - distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
2024-07-26 11:37:05,244 - distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
2024-07-26 11:37:12,850 - distributed.utils_perf - WARNING - full garbage collections took 15% CPU time recently (threshold: 10%)
2024-07-26 11:37:15,812 - distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
2024-07-26 11:37:23,458 - distributed.utils_perf - WARNING - full garbage collections took 15% CPU time recently (threshold: 10%)
2024-07-26 11:37:29,533 - distributed.utils_perf - WARNING - full garbage

TimeoutError: timed out after 60 s.

2024-07-26 12:04:02,292 - distributed.scheduler - ERROR - Workers ['tcp://127.0.0.1:34119'] did not shut down within 30s; force closing
2024-07-26 12:04:02,372 - distributed.scheduler - ERROR - 1/1 nanny worker(s) did not shut down within 30s: {'tcp://127.0.0.1:34119'}
Traceback (most recent call last):
  File "/ocean/projects/phy210048p/malanche/ztfdr17-psdr2-gaiadr3/cenv/lib/python3.12/site-packages/distributed/utils.py", line 832, in wrapper
    return await func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/ocean/projects/phy210048p/malanche/ztfdr17-psdr2-gaiadr3/cenv/lib/python3.12/site-packages/distributed/scheduler.py", line 6377, in restart_workers
    raise TimeoutError(
TimeoutError: 1/1 nanny worker(s) did not shut down within 30s: {'tcp://127.0.0.1:34119'}
2024-07-26 12:04:02,373 - distributed.scheduler - ERROR - 1/1 nanny worker(s) did not shut down within 30s: {'tcp://127.0.0.1:34119'}
Traceback (most recent call last):
  File "/ocean/projects/phy210048

In [9]:
%%time

dataset = pyarrow.dataset.dataset(catalog_name).sort_by('oid')
pa.dataset.write_dataset(
    dataset,
    f"{catalog_name}_sorted",
    format='parquet',
    partitioning=pyarrow.dataset.partitioning(
        flavor="filename",
        schema=pyarrow.schema([dataset.schema.field('fieldid')]),
    ),
)

CPU times: user 4min 44s, sys: 11min 58s, total: 16min 42s
Wall time: 7min 32s
